In [30]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos
import numba as nb
from scipy.optimize import fsolve
from numpy.linalg import inv

In [9]:
# power flow function (equality constraints)
def gfun(x, u, p):
  
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    F1 = 4.0*VM2*VM2 + VM2*VM3*(-4*cos(VA23) + 10*sin(VA23)) - P2
    F2 = (8.0*VM3*VM3 + VM3*VM1*(-4*cos(VA31) + 5*sin(VA31))
          + VM3*VM2*(-4*cos(VA32) + 10*sin(VA32)) + P3)
    F3 = (15.0*VM3*VM3 + VM3*VM1*(-4*sin(VA31) - 5*cos(VA31))
          + VM3*VM2*(-4*sin(VA32) - 10*cos(VA32)) + Q3)

    return np.array([F1, F2, F3])

# cost function
def cfun(x, u, p):

    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0

    cost = (w1*(4.0*VM1*VM1 + VM1*VM3*(-4*cos(VA13) + 5*sin(VA13))) +
          w2*P2)
    
    return cost

In [16]:
# Jacobians and gradients

def gfun_x(x, u, p):

    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((3, 3))
    
    #F1
    J[0, 0] =  VM2*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[0, 1] =  VM2*VM3*(-4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    J[0, 2] =  VM2*VM3*(4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM1*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3)) + 16.0*VM3
    J[1, 1] =  VM1*VM3*(-4*sin(VA1 - VA3) + 5*cos(VA1 - VA3)) + VM2*VM3*(-4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    J[1, 2] =  VM2*VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    #F3
    J[2, 0] =  VM1*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3)) + VM2*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3)) + 30.0*VM3
    J[2, 1] =  VM1*VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[2, 2] =  VM2*VM3*(10*sin(VA2 - VA3) + 4*cos(VA2 - VA3))

    return J

def gfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((3, 3))
    
    #F1
    J[0, 0] =  0
    J[0, 1] =  -1
    J[0, 2] =  8.0*VM2 + VM3*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[1, 1] =  0
    J[1, 2] =  VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F3
    J[1, 0] =  VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[1, 1] =  0
    J[1, 2] =  VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))

    return J


def cfun_x(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  VM1*w1*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    grad[1] =  VM1*VM3*w1*(-4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    grad[2] =  0
    
    return grad

def cfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  w1*(8.0*VM1 + VM3*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)))
    grad[1] =  w2
    grad[2] =  0
    
    return grad

Initialize script with same initial conditions as in the paper

In [3]:
# initial parameters
x = np.zeros(3)
u = np.zeros(3)
p = np.zeros(3)

# this is an initial guess
x[0] = 1.0 #VM3
x[1] = 0.0 #VA3
x[2] = 0.0 #VA2

# this is given by the problem data, but might be "controlled" via OPF
u[0] = 1.0 #VM1
u[1] = 1.7 #P2
u[2] = 1.0 #VM2

# these parameters are fixed through the computation
p[0] = 0.0 #VA1, slack angle
p[1] = 2.0 #P3
p[2] = 1.0 #Q3

# print initial guesses
print(x)
print(u)

[1. 0. 0.]
[1.  1.7 1. ]


In [25]:
# POWER FLOW ALGO

def powerflow(x, u, p):
    
    sol = fsolve(gfun, x, args=(u,p,))
    return sol
    
print(powerflow(x, u, p))

[ 0.88186783 -0.00094814  0.1349708 ]


In [52]:
# Reduced gradient iteration

max_iter = 300
xk = np.copy(x)
uk = np.copy(u)

for i in range(max_iter):
    
    # power flow
    xk = powerflow(xk, uk, p)

    # lambda calculation
    J_x = gfun_x(xk, uk, p)
    G_x = cfun_x(xk, uk, p)

    lam = -np.dot(inv(np.transpose(J_x)), G_x)

    # gradient cost function
    J_u = gfun_u(xk, uk, p)
    G_u = cfun_u(xk, uk, p)

    grad_c = G_u + np.dot(np.transpose(J_u), lam)
    print("Norm of gradient: ", np.linalg.norm(grad_c))
    
    # evaluate cost function
    print("Cost function: ", cfun(xk, uk, p))    
    
    # compute step
    alpha = 0.08
    uk = uk - alpha*grad_c

Norm of gradient:  0.6806350735577951
Cost function:  2.176710951198614
Norm of gradient:  0.5208553342382815
Cost function:  2.1795465377422643
Norm of gradient:  0.5301385380257911
Cost function:  2.195685078446081
Norm of gradient:  0.5310377877882734
Cost function:  2.2141979269924628
Norm of gradient:  0.5270877786707068
Cost function:  2.236391035785762
Norm of gradient:  0.5135887282073747
Cost function:  2.263130479743788
Norm of gradient:  0.4842284003188323
Cost function:  2.295526425593832
Norm of gradient:  0.42879188346856134
Cost function:  2.3344751241358668
Norm of gradient:  0.3348081832575279
Cost function:  2.3794545825224938
Norm of gradient:  0.20308910140753886
Cost function:  2.4253478730492573
Norm of gradient:  0.09068333078146222
Cost function:  2.4592807927826703
Norm of gradient:  0.06752092366428539
Cost function:  2.4713977136177006
Norm of gradient:  0.06492193323234306
Cost function:  2.4720426678475347
Norm of gradient:  0.06246097226096025
Cost functio

Norm of gradient:  3.0167966775379523e-05
Cost function:  2.4661140004808715
Norm of gradient:  2.899224959741075e-05
Cost function:  2.4661139563780097
Norm of gradient:  2.786200977626522e-05
Cost function:  2.4661139137320305
Norm of gradient:  2.677615042264638e-05
Cost function:  2.4661138729950745
Norm of gradient:  2.5732316360981195e-05
Cost function:  2.4661138336216037
Norm of gradient:  2.472944779621423e-05
Cost function:  2.466113795994003
Norm of gradient:  2.3765412856189652e-05
Cost function:  2.466113759641048
Norm of gradient:  2.2839192742240387e-05
Cost function:  2.4661137248857603
Norm of gradient:  2.194885557656327e-05
Cost function:  2.466113691320852
Norm of gradient:  2.1093426195337742e-05
Cost function:  2.466113659218869
Norm of gradient:  2.027115208260192e-05
Cost function:  2.466113628227539
Norm of gradient:  1.9481103315631094e-05
Cost function:  2.4661135985765195
Norm of gradient:  1.8721688456395563e-05
Cost function:  2.466113569960891
Norm of gra